In [30]:
!kaggle competitions download -c spaceship-titanic

spaceship-titanic.zip: Skipping, found more recently modified local copy (use --force to force download)


In [31]:
import zipfile
import os
import pandas as pd

# 解压数据集
path = "spaceship-titanic"
if not os.path.exists(path=path):
    os.makedirs(name=path)
fp = zipfile.ZipFile("spaceship-titanic.zip", 'r')
fp.extractall(path)

# 读取数据
train_csv = pd.read_csv(filepath_or_buffer=f"{path}/train.csv")
tests_csv = pd.read_csv(filepath_or_buffer=f"{path}/test.csv")
train_csv

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [32]:
# 初步组合数据
all_features = pd.concat((train_csv.iloc[1:, 1:-2], tests_csv.iloc[1:, 1:-1])) # 删除name和labels已经PassengerId等无关特征
all_features[1:3]

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0


In [33]:
import matplotlib.pyplot as plt
from pandas import Series

def groupby_cate(data: Series) -> dict[str, float]:
    """统计离散数据频数

    Args:
        data (list): _description_

    Returns:
        dict[str, float]: _description_
    """
    cate_frequent = {}
    for cate in data:
        if cate not in cate_frequent:
            cate_frequent[cate] = 1
        else:
            cate_frequent[cate] = cate_frequent[cate] + 1
    return cate_frequent

In [34]:
# 分析特征-HomePlanet 将nan设置为Earth 采用one-hot encoding处理本列数据
all_features['HomePlanet'] = all_features['HomePlanet'].fillna('Earth')
print(groupby_cate(all_features['HomePlanet']))

{'Earth': 7152, 'Europa': 3132, 'Mars': 2684}


In [ ]:
# 分析特征-CryoSleep（冷冻睡眠是否与客房服务有关、冷冻睡眠是否与出发地有关、冷冻睡眠大概率与消费金额有关）
null_cryosleep_index = (all_features['CryoSleep'].isnull()).index
# zero_expenses_null_cryosleep_index = list(
#     set((all_features['RoomService'][null_cryosleep_index] == 0).index) &
#     set((all_features['FoodCourt'][null_cryosleep_index] == 0).index) & 
#     set((all_features['ShoppingMall'][null_cryosleep_index] == 0).index) &
#     set((all_features['Spa'][null_cryosleep_index] == 0).index) &
#     set((all_features['VRDeck'][null_cryosleep_index] == 0).index)
# )
# len(zero_expenses_null_cryosleep_index)

8692